## Step 1: Install dependencies

### Create virtual envirnment

conda create -n yourenvname python=x.x

!conda create -n RASA_X_venv python=3.6
<br>
!conda activate RASA_X_venv

#### To add the environment in Jupyter Notebook (This is optional)
!conda activate RASA_X_venv<br>
!pip install ipykernel<br>
!python -m ipykernel install --user --name RASAXvenv --display-name "Python (RASAXvenv)"

#### Install RASA X
This will install required files of about ~300MBs and should take about 10 to 15 min depending on network speed

!pip install rasa-x --extra-index-url https://pypi.rasa.com/simple

#### Install RASA SDK to write custom actions in python
!pip install rasa-sdk

#### Install spacy to use pre-trained models and other functions
!pip install spacy<br>
!python -m spacy download en_core_web_sm

#### Link the spacy model
!python -m spacy link en_core_web_sm en

## Step 2: Build a Chatbot

### Define intent

Intent is understanding what the user wants to say. For example — if the user says “Reserve a table at Cliff House tonight” the intent can be classified as to reserve/book the table.

For the current use case, we want our bot to be able to understand following 5 intents from the customers among other courteous ones.
1. withdraw_money
2. check_balance
3. deposite_money
4. request_cheque_book
5. transfer_money

In [29]:
nlu_md = """

## intent:greet
- hey. I am [James](PERSON)
- hello. I am [William](PERSON)
- hi. I am [Kabir](PERSON)
- Hi, I am [Salman](PERSON)
- Hi, I am [Brad](PERSON)
- Hi, I am [Tom](PERSON)
- Hi, I am [Bill](PERSON)
- Hi, I am [William](PERSON)

## intent:fine_ask
- I am good, how are you doing?
- I'm fine, how are you?
- I'm good, how are you?

## intent:thanks
- Thanks
- Thank you so much

## intent:goodbye
- Bye for now
- Goodbye
- Thanks for help. Bye
- No, I am good as of now. Bye
- Bye
- Bbye

## intent:fine_normal
- I am doing great
- I'm doing great
- I'm fine
- I'm good

## intent:check_balance
- What is my account balance?
- How much I have?
- What is my balance?
- My account balance? 

## intent:deposite_money
- I would like to deposite money
- want to deposite
- I want to deposite money in my account

## intent:request_cheque_book
- Need cheque book
- Want a cheque book
- Can I order a cheque book please?
- Send me a cheque book

## intent:transfer_money
- transfer [$500](amount)
- transfer money to someone
- send [$500](amount) to James
- give money to someone
- I want to transfer [$1000](amount)
- I want to send some amount

## intent:withdraw_money
- I want to get [$500](amount)
- I would like to withdraw [$10](amount)
- Please send me [$30](amount)
- withdraw [$385](amount) and send me to my home

"""
%store nlu_md >data/nlu.md

Writing 'nlu_md' (str) to file 'data/nlu.md'.


### Define configurations and Policies

The rasa core policies decide which action to take at every step in the conversation. There are different policies to choose from, and one can include multiple policies in a single rasa core Agent. But at every turn, the policy which predicts the next action with the highest confidence is used. We have configured a basic policy for our bot as shown below which has <b>FallbackPolicy</b> as well. The fallback policy comes in to picture when <b>‘nlu_threshold’ & ‘core_threshold’</b> meets the levels defined in the policy which means that bot is not able to understand the user message and it responds with <b>‘utter_default’</b>.

<b>KerasPolicy</b> uses a neural network implemented in Keras to select the next action. The default architecture is based on an LSTM (Long Short Term Memory) model

<b>MemoizationPolicy</b> memorizes the conversations in your training data. It predicts the next action with confidence 1.0 if this exact conversation exists in the training data, otherwise, it predicts ‘None’ with confidence 0.0

<b>FallbackPolicy</b> invokes a fallback action if the intent recognition has confidence below nlu_threshold or if none of the dialogue policies predict action with confidence higher than core_threshold

One important hyperparameter for Rasa Core policies is the <b>max_history</b>. This controls how much dialogue history the model looks at to decide which action to take next

In [28]:
config = """
# Configuration for Rasa NLU.
# https://rasa.com/docs/rasa/nlu/components/
language: en

pipeline:
- name: "SpacyNLP"                  # loads the spacy language model
- name: "SpacyTokenizer"            # splits the sentence into tokens
- name: "SpacyFeaturizer"           # transform the sentence into a vector representation
- name: "SpacyEntityExtractor"
- name: "SklearnIntentClassifier"   # uses the vector representation to classify using SVM
- name: "CRFEntityExtractor"
- name: "EntitySynonymMapper"       # trains the synonyms

# Configuration for Rasa Core.
# https://rasa.com/docs/rasa/core/policies/

policies:
  - name: KerasPolicy
    epochs: 100
  - name: MemoizationPolicy
    max_history: 5
  - name: FallbackPolicy
    fallback_action_name: 'utter_default'
    nlu_threshold: 0.1
    core_threshold: 0.2
  - name: MappingPolicy
  - name: FormPolicy

""" 
%store config >config.yml

Writing 'config' (str) to file 'config.yml'.


### Write  Stories

<b>stories_md</b> define the sample interaction between the user and chatbot in terms of intent and action taken by the bot. Like in the previous example bot got the intent of booking the table and entities like place and time but still, there is an entity missing — no of people and that would make the next action from the bot.

In [23]:
stories_md = """

## fallback
- utter_default

## greeting path 1
* greet{"PERSON":"Will"}
  - slot{"PERSON": "Will"}
  - utter_greet

## fine path 1
* fine_normal
  - utter_help

## fine path 2
* fine_ask
  - utter_reply

## thanks path 1
* thanks
  - utter_anything_else

## happy path
* greet{"PERSON":"Will"}
  - slot{"PERSON": "Will"}
  - utter_greet

## say goodbye
* goodbye
  - utter_goodbye

## check balance
* check_balance
  - utter_check_balance

## deposite money
* deposite_money
  - utter_deposite_money

## request cheque book
* request_cheque_book
  - utter_request_cheque_book

## transfer money
* transfer_money
  - utter_transfer_money

## withdraw money
* withdraw_money
  - utter_withdraw_money

"""

%store stories_md >data/stories.md

Writing 'stories_md' (str) to file 'data/stories.md'.


### Definine a Domain

### Domain(domain.yml)
The domain consists of five key parts consisting of intents, entities, slots, actions, and templates.

<b>slots:</b> Slots are basically bot’s memory. They act as a key-value store which can be used to store information the user provided (e.g their home city) as well as information gathered about the outside world (e.g. the result of a database query).

<b>entity:</b> Entity is to extract the useful information from the user input. From the example above <i>“Reserve a table at Cliff House tonight”</i> the entities extracted would be place and time. Place — Cliff House and Time — tonight.

<b>intent: </b> Intent is understanding what the user wants to say. For example — if the user says <i>“Reserve a table at Cliff House tonight”</i> the intent can be classified as to reserve/book the table.

<b>actions:</b> Actions are bots response to user input. There are 3 kinds of actions in Rasa Core: <b>default actions, utter actions & custom actions</b>

<b>templates:</b> templates define the actual text responses used by the dialogue engine. The engine will pick one random response out of all the options. Notice that <b>utter_its_nice_to_meet_you</b> uses <b>PERSON</b> slot in the response to personalize it.

In [21]:
domain_yml = """

slots:
  PERSON:
    type: text
    auto_fill: true
  account_number:
    type: unfeaturized
    auto_fill: false
  account_balance:
    type: unfeaturized
    auto_fill: false
  tpin:
    type: unfeaturized
    auto_fill: false
  amount:
    type: unfeaturized
    auto_fill: false

entities:
- account_number
- account_balance
- tpin
- amount
- PERSON

intents:
- greet
- fine_ask
- thanks
- goodbye
- fine_normal
- check_balance
- deposite_money
- request_cheque_book
- transfer_money
- withdraw_money

actions:
- utter_default
- utter_greet
- utter_help
- utter_reply
- utter_anything_else
- utter_goodbye
- utter_check_balance
- utter_deposite_money
- utter_request_cheque_book
- utter_transfer_money
- utter_withdraw_money

templates:
  utter_default:
    - text: I am not sure what you're aiming for. Let's try again
    - text: I am sorry but I am not able to get you. Let's try again
    - text: My appologies but I am not able to get you. Let's try again
  utter_greet:
    - text: Hey {PERSON}, how are you?
    - text: Hello {PERSON}, How are you doing?
  utter_help:
    - text: Great{PERSON}. How can I help you?
    - text: Great. Tell me How can I help you?
  utter_reply:
    - text: I'm doing great. Please let me know what I can do for you.
    - text: I'm doing great. Tell me How can I help you today?
  utter_anything_else:
    - text: No worries. Is there anything else I can help you with?
    - text: No worries. Let me know if there is anything else I can help you with
  utter_goodbye:
    - text: It was a pleasure to help you
    - text: Hope it was helpful
    - text: Bye and have a nice day
    - text: Bbye and have a nice day
  utter_check_balance:
    - text: Your current balance is $$$$.
    - text: Your have $$$$ in your account. 
  utter_deposite_money:
    - text: Your balance after deposite of $$$$ is $$$$. 
    - text: Your balance is $$$$ after the deposite.     
  utter_request_cheque_book:
    - text: Thank you for your request. Your cheque book will be delivered at your registered address. 
    - text: We will send your cheque book at your registered address.     
  utter_transfer_money:
    - text: An amount of $$$$ has been transferred to target_account. Your current balance is $$$$. 
    - text: We have transferred $$$$ to target_account. Your current balance is $$$$.     
  utter_withdraw_money:
    - text: An amount of $$$$ will be delivered at your address by tomorrow. Please share your transaction pin to validate transaction.
    - text: We are sending money your way. An amount of $$$$ will be delivered by tomorrow.
"""

%store domain_yml >domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


### Validate Data
Run below line in terminal window to see errors in data. Make sure you are in the same directory where RASA project was initiated.

!rasa data validate

### Training

Use !rasa train to train overall core and nlu models

Use below functions to train only NLU part

In [10]:
from rasa_nlu.training_data import load_data
from rasa_nlu.model import Trainer
from rasa_nlu import config

training_data = load_data("data/nlu.md")
trainer = Trainer(config.load("config.yml"))
trainer.train(training_data)
model_directory = trainer.persist("./models", fixed_model_name="nlu")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.








Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\ASUS\Anaconda3\envs\RASA_x_venv\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\ASUS\Anaconda3\envs\RASA_x_venv\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\ASUS\Anaconda3\envs\RASA_x_venv\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\ASUS\Anaconda3\envs\RASA_x_venv\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0

### Evaluating your bot


In [30]:
# !rasa test nlu -u bankingbot/data/nlu.md --config bankingbot/config.yml --cross-validation

from rasa_nlu.test import run_evaluation

run_evaluation("data/nlu.md", model_directory)

100%|█████████████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 198.30it/s]
C:\Users\ASUS\Anaconda3\envs\RASA_x_venv\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\ASUS\Anaconda3\envs\RASA_x_venv\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\ASUS\Anaconda3\envs\RASA_x_venv\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


{'intent_evaluation': {'predictions': [{'text': 'hey. I am James',
    'intent': 'greet',
    'predicted': 'goodbye',
    'confidence': 0.252955574236716},
   {'text': 'hello. I am William',
    'intent': 'greet',
    'predicted': 'goodbye',
    'confidence': 0.2534159238804949},
   {'text': 'hi. I am Kabir',
    'intent': 'greet',
    'predicted': 'goodbye',
    'confidence': 0.26322258140524246},
   {'text': 'Hi, I am Salman',
    'intent': 'greet',
    'predicted': 'goodbye',
    'confidence': 0.23023327873648156},
   {'text': 'Hi, I am Brad',
    'intent': 'greet',
    'predicted': 'goodbye',
    'confidence': 0.23465581409771924},
   {'text': 'Hi, I am Tom',
    'intent': 'greet',
    'predicted': 'goodbye',
    'confidence': 0.2702636157930516},
   {'text': 'Hi, I am Bill',
    'intent': 'greet',
    'predicted': 'goodbye',
    'confidence': 0.22512963801241742},
   {'text': 'Hi, I am William',
    'intent': 'greet',
    'predicted': 'goodbye',
    'confidence': 0.266179931536311

### Visualize story flow

Run below line in terminal window to see story flow. Make sure you are in the same directory where RASA project was initiated.

!rasa visualize

### Talk to your bot

Type below command in terminal to talk to your bot

!rasa shell